# Libraries

In [134]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd


from scipy import signal
from scipy.io import wavfile

# from pybv import write_brainvision
from pyprep.prep_pipeline import PrepPipeline
from mne_icalabel import label_components

# set working directory
work_dir = '/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/'

# Trigger lag fix

## parameters

In [13]:
# directory
input_dir = work_dir + 'data_raw/'
output_dir = work_dir + 'data_preprocessed/1_trigger_lag_fixed/'
# create a folder if the folder doesn't exist
os.makedirs(output_dir, exist_ok=True)

# create a dictionary for blocks and markers
block_dict = {
    'ChEn':1000, # standard: chinese; deviant: english
    'EnCh':2000, # standard: english; deviant: chinese
    'InEn':3000, # standard: indian; deviant: english
    'EnIn':4000, # standard: english, deviant: indian
}


# exclude participants with missing data
exclude_ppts = [
    '140', # no data
    '197', # no data
    
    '135', # incomplete data
    '179', # incomplete data
    '182', # incomplete data

    '141', # wrong experiment was run

    '261', # no StimTrak recording

    '115', # unknown
]

# trigger searching window (actual trigger time based on audio - trigger time in the data)
t_left = -0.01
t_right = 0.05

In [14]:
#######################################################
#### create a dictionary for trigger codes and descriptions ####
df = pd.read_csv("mapping.txt", delimiter='\t')
mapping = dict(zip(df['code'], df['description']))
#######################################################

#### create a trigger dictionary for each stim's standard version and deviant version ####

# intialize a dictionary for triggers
trigger_dict = {}

# read in trigger_codes file
with open('trigger_codes.txt','r') as tf:
    for line in tf:
        # read in the current line
        line = line.replace('\n','')
        # separate fileNames and triggerMarker
        label, marker = line.split('\t')
        # convert trigger markers to integer
        marker = int(marker)
        # create label for stims used as standards
        trigger_dict[marker] = label + '-s'
        # add 100 for deviant marker
        marker_deviant = marker + 100
        # create label for stims used as deviants
        trigger_dict[marker_deviant] = label + '-d'



#### trigger correction for each file ####

# initialize a dictionary to save bad stims
all_bad_stim_dict = {}

# read in eeg data files
all_files = os.listdir(input_dir)

#### for each file, create an all_block dictionary to store each block and the indices of trials of that block  ####
for file in all_files:
    if file.endswith(".vhdr") and (file.split('.')[0]+ '_corr.fif' not in os.listdir(output_dir)):

        # exclude bad data
        if (file.split('.')[0].split('_')[1] not in exclude_ppts):
            continue
            
        # read in vhdr files
        raw = mne.io.read_raw_brainvision(input_dir + file, preload = True)

        # extract sampling rate
        eeg_sfreq = raw.info['sfreq']
        
        # If the aux channel is not named 'StimTrak', change the channel name to 'StimTrak'
        if raw.info['ch_names'][31] != 'StimTrak':
            # change the channel name
            raw.rename_channels({raw.info['ch_names'][31]: 'StimTrak'})
            # specify the audio channel
            raw.set_channel_types({'StimTrak':'misc'})

        ##########################################################################################
        #### get trigger code, audio data, create a trigger dictionary for each stim's standard version and deviant version ####

        # initialize dictionaries
        audio = {}
        lengths = {}
        
        # del trigger_dict[99999]
        for marker,label in trigger_dict.items():
            # extract file name
            file_name = label[:7]
            # if not already in audio dictionary, get the info of the audio file
            if file_name not in audio:
                # get sample rate and data of the audio file
                sampleRate, data = wavfile.read(work_dir + 'experiment_program/stimuli/{}.wav'.format(file_name))
                # calculate sound file length
                lengths[file_name] = len(data)/sampleRate
                # reduce the sampling rate of the audio file by the factor of int(sampleRate/eeg_freq)
                data_downsampled = signal.decimate(data,int(sampleRate/eeg_sfreq), ftype='fir')
                # add info the audio dictionary
                audio[file_name] = data_downsampled

        #### making events ####
        # for each stimulus, mark the block info
        events_from_annot, event_dict = mne.events_from_annotations(raw, verbose='WARNING')
        # remove useless markers (New Segment: 99999, pause: 222, and continue: 223)
        events_from_annot = events_from_annot[events_from_annot[:, 2] < 200]
                
        # initialize the train for each standards+deviant sequence
        train = np.array([]).astype(int)
        
        all_block = {}
        isStandard = True # whether the standard in a train has been noted
            
        # loop over each trigger
        for i in range(events_from_annot.shape[0]):
                
            # add the current token to the train
            train = np.append(train,i)

            # trigger code
            trigger_marker = events_from_annot[i,2]
            
            # if the trigger code is smaller than 100 and the standard note is true
            if (trigger_marker<100) & isStandard:
                # save the standard stim category
                block = trigger_dict[trigger_marker][:2]
                # toggle standard note
                standard = False

            # if the trigger value is over 100
            elif trigger_marker>100:
                # append the deviant stim category 
                block = block + trigger_dict[trigger_marker][:2]

                # if the block is not present in all block
                if block not in all_block:
                    # add the new block and the token idx
                    all_block[block] = train[2:] # [2:] to exclude the first 2 standards
                else:
                    # add the token idx to the existing block
                    all_block[block] = np.concatenate([ all_block[block], train[2:] ], axis=None)

                # reset train
                train = np.array([]).astype(int)

                # toggle standard note
                isStandard = True
        
        # loop over each block and its trigger index
        for k,v in all_block.items(): 
            # recode the trigger value to reflect block and stim category
            events_from_annot[tuple(v),2] += block_dict[k]

        # exclude the first 2 standards
        events_from_annot = events_from_annot[events_from_annot[:,2]>1000]
        ##########################################################################################
        
        ########################################################
        #### calculate cross correction to detect the delay ####
        
        # initialize delay list
        delays = np.array([])
        # initialize bad stim list
        bad_stim = []
        corr_results = []
        
        # loop over each event
        for i in range(events_from_annot.shape[0]):

            # get current event info [time, duration, annotation]
            event = events_from_annot[i]
            # get the onset latency (s) of the event
            time = event[0]/eeg_sfreq
            # get the file name of the event
            name = trigger_dict[event[2]%100].split('-')[0]
            # get the data from the sound channel
            audio_eeg = raw.get_data(
                picks = ['StimTrak'],
                tmin = time + t_left,
                tmax = time + lengths[name] + t_right,
            )[0]
            # actual stimulus
            audio_stim = audio[name]
            
            # Z-score normalization (subtract mean, divide by std)
            audio_eeg = (audio_eeg - np.mean(audio_eeg)) / np.std(audio_eeg)
            audio_stim = (audio_stim - np.mean(audio_stim)) / np.std(audio_stim)

            # cross-correlation
            corr = signal.correlate(audio_eeg, audio_stim, mode='full')
            # Normalize cross-correlation
            corr = corr / (np.linalg.norm(audio_eeg) * np.linalg.norm(audio_stim))
            # Find peak correlation value
            max_corr = np.max(corr)    
            
            # if the maximum correction (sum of products) is less than a threshold (empirical, but 0.5 is good for most cases)
            if max_corr < 0.5:
                # mark the stim bad
                bad_stim.append(i)
            
            # append the maximum correlation
            corr_results.append(max_corr)
            
            # the lags for cross-correlation
            lags = signal.correlation_lags(
                audio_eeg.size,
                audio_stim.size,
                mode="full")
            # get the lag of the maximum cross-correlation
            lag = lags[np.argmax(corr)] + t_left*eeg_sfreq
            
            # save the lag for non-starting events
            delays = np.append(delays, lag)
        ########################################################

        ##################################################################################################
        #### plot the wave from the stim track and the eeg channel of the token with the minimum corr ####
        
        min_corr = np.argmin(corr_results)
        # get current event info [time, duration, annotation]
        event = events_from_annot[min_corr]
        # get the onset latency (s) of the event
        time = event[0]/eeg_sfreq
        # get the file name of the event
        name = trigger_dict[event[2]%100].split('-')[0]
        # get the data from the sound channel
        audio_eeg = raw.get_data(
            picks = ['StimTrak'],
            tmin = time + t_left,
            tmax = time + lengths[name] + t_right,
        )[0]
        # actual stimulus
        audio_stim = audio[name]
        # Z-score normalization (subtract mean, divide by std)
        audio_eeg = (audio_eeg - np.mean(audio_eeg)) / np.std(audio_eeg)
        audio_stim = (audio_stim - np.mean(audio_stim)) / np.std(audio_stim)
        # plot
        fig, ax = plt.subplots()
        ax.plot(audio_eeg, label = 'StimTrak', alpha = 0.6)
        ax.plot(audio_stim, label = 'wave', alpha = 0.6)
        ax.set_title(file)
        ax.legend()
        fig.savefig(output_dir + file.split('.')[0] + "_minCor.png", dpi=300, bbox_inches='tight')
        ##################################################################################################

                
        ####################################################
        #### correct for trigger lag and save log files ####

        # add number of bad stim info
        all_bad_stim_dict[file] = len(bad_stim)

        # remove items associated with bad stims from the event list
        events_from_annot = np.delete(events_from_annot, bad_stim, 0)
        
        # remove items associated with bad stims from the delay list
        delays = np.delete(delays, bad_stim, 0)

        # add delay back to the onset latency of each event
        events_from_annot[:,0] = events_from_annot[:,0] + delays

        # # convert individual event marker to conditions
        # events_from_annot[:,2] = events_from_annot[:,2] - events_from_annot[:,2]%100

        # create annotations
        annot_from_events = mne.annotations_from_events(
            events = events_from_annot,
            event_desc = mapping,
            sfreq = raw.info['sfreq']
        )

        # set annotations
        raw.set_annotations(annot_from_events)
        
        # drop the audio channel in data
        raw.drop_channels(['StimTrak'])
        
        # save as a file-into-file data
        raw.save(output_dir + file.split('.')[0]+ '_corr.fif')
        
        # save lag data
        np.savetxt(output_dir + file.replace('.vhdr', '_delays.txt'), delays, fmt='%i')
        ####################################################
        

# save the number of bad stims of all participant
with open(output_dir + 'bad_stim.txt', 'a') as f:
    for key, value in all_bad_stim_dict.items():
        if value > 0:
            f.write(key + '\t' + str(value) + '\n')

# Bad channel correction
- filtering
- resampling
- remove line noise
- bad channel detection & repairing
- add back reference channel TP9

## parameters

In [12]:
# set directory
input_dir = work_dir + 'data_preprocessed/1_trigger_lag_fixed/'
output_dir = work_dir + 'data_preprocessed/2_bad_channel_corrected/'
# create a folder if the folder doesn't exist
os.makedirs(output_dir, exist_ok=True)

# filter cutoff frequencies (low/high)
f_low = 1
f_high = 100

# resampling frequency
f_res = 250

# line frequency
line_freq = 60

# preprocessing parameters
prep_params = {
    "ref_chs": 'eeg',
    "reref_chs": 'eeg', # average re-reference
    "line_freqs": np.arange(line_freq, f_res/2, line_freq),
}

# create a montage file for the pipeline
montage = mne.channels.make_standard_montage("standard_1020")

# interpolation method
# method=dict(eeg="spline")

In [5]:
#####################################################
#### Preprocessing (filtering, resampling, bad channel detection/interpoloation, re-reference) ####
#####################################################

# get all file namesin the folder
all_input = os.listdir(input_dir)
all_output = os.listdir(output_dir)


for file in all_input:
    if file.endswith("corr.fif") & (file.split('.')[0]+ '_prep.fif' not in all_output):
        
        # read in file
        raw = mne.io.read_raw_fif(input_dir + file, preload=True)
        
        # set channel type
        raw.set_channel_types({'Fp1':'eog', 'Fp2':'eog'})

        # filter
        raw.filter(l_freq = f_low, h_freq = f_high)
        
        #### cut off the beginning and ending part ####
        
        # get the onset of the first and the last event ####
        events_from_annot, event_dict = mne.events_from_annotations(raw, verbose='WARNING')

        # define the beginning time (in seconds)
        crop_start = events_from_annot[0][0]/raw.info['sfreq'] - 10

        # define the ending time (in seconds)
        crop_end = events_from_annot[-1][0]/raw.info['sfreq'] + 10

        # crop the data
        raw.crop(
            tmin=max(crop_start, raw.times[0]), 
            tmax=min(crop_end, raw.times[-1])
        )
        
        # resample
        raw.resample(sfreq = f_res)

        # read in channel location info
        raw.set_montage(montage)
        
        ####  Use PrePipeline to preprocess ####
        '''
        1. detect and interpolate bad channels
        2. remove line noise
        3. re-reference
        '''
        
        # apply pyprep
        prep = PrepPipeline(raw, prep_params, montage, random_state=42)
        prep.fit()

        
        # export a txt file for the interpolated channel info
        with open(output_dir + 'bad_channel.txt', 'a+') as f:
            _ =f.write(
                file + ':\n' +
                "- Bad channels original: {}".format(prep.noisy_channels_original["bad_all"]) + '\n' +
                "- Bad channels after robust average reference: {}".format(prep.interpolated_channels) + '\n' +
                "- Bad channels after interpolation: {}".format(prep.still_noisy_channels) + '\n'
            )

        # save the pypred preprocessed data into the raw data structure
        raw = prep.raw

        # spline interpolating remaining bad channels if any
        raw.interpolate_bads()

        # add back the reference channel
        raw = mne.add_reference_channels(raw,'TP9')

        # add the channel loc info (for the newly added reference channel)
        raw.set_montage(montage)
        
        # save
        raw.save(output_dir + file.split('.')[0]+ '_prep.fif')

### test

In [ ]:
# read in file
raw = mne.io.read_raw_fif(input_dir + file, preload=True)

# set channel type
raw.set_channel_types({'Fp1':'eog', 'Fp2':'eog'})

# filter
raw.filter(l_freq = f_low, h_freq = f_high)

#### cut off the beginning and ending part ####

# get the onset of the first and the last event ####
events_from_annot, event_dict = mne.events_from_annotations(raw, verbose='WARNING')

# define the beginning time (in seconds)
crop_start = events_from_annot[0][0]/raw.info['sfreq'] - 10

# define the ending time (in seconds)
crop_end = events_from_annot[-1][0]/raw.info['sfreq'] + 10

# crop the data
raw.crop(
    tmin=max(crop_start, raw.times[0]), 
    tmax=min(crop_end, raw.times[-1])
)

# resample
raw.resample(sfreq = f_res)

# read in channel location info
raw.set_montage(montage)

####  Use PrePipeline to preprocess ####
'''
1. detect and interpolate bad channels
2. remove line noise
3. re-reference
'''

# apply pyprep
prep = PrepPipeline(raw, prep_params, montage, random_state=42)
prep.fit()


# export a txt file for the interpolated channel info
with open(output_dir + 'bad_channel.txt', 'a+') as f:
    _ =f.write(
        file + ':\n' +
        "- Bad channels original: {}".format(prep.noisy_channels_original["bad_all"]) + '\n' +
        "- Bad channels after robust average reference: {}".format(prep.interpolated_channels) + '\n' +
        "- Bad channels after interpolation: {}".format(prep.still_noisy_channels) + '\n'
    )

# save the pypred preprocessed data into the raw data structure
raw = prep.raw



# add back the reference channel
raw = mne.add_reference_channels(raw,'TP9')

# ICA bad trial correction

## parameters

In [8]:
# directory
input_dir = work_dir + 'data_preprocessed/2_bad_channel_corrected/'
output_dir = work_dir + 'data_preprocessed/3_ica/'

# create a folder if the folder doesn't exist
os.makedirs(output_dir, exist_ok=True)

# run ica on epoched data or continuous data
# ica_input_type = 'epoch'
ica_input_type = 'continuous'

# up to which IC you want to consider
ic_upto = 15
# ic_upto = 99

# Epoch window: 
erp_t_start = -0.2; erp_t_end = 0.8 # ERP: -100 ~ 700 ms
baseline = (-0.2, 0)

In [7]:
# get all file names in the folder
all_input = os.listdir(input_dir)
all_output = os.listdir(output_dir)


# for each file
for file in all_input:
    if file.endswith("prep.fif") and (file.split('.')[0]+ '_ica.fif' not in all_output): 
    
        # read in file
        raw = mne.io.read_raw_fif(input_dir + file, preload=True)
        
        # make a filtered file copy ICA. It works better on signals with 1 Hz high-pass filtered and 100 Hz low-pass filtered
        raw_filt = raw.copy().filter(l_freq = 1, h_freq = 100)
    
        # apply a common average referencing, to comply with the ICLabel requirements
        raw_filt.set_eeg_reference("average")

        # initialize ica components enough components to explain 0.999999 of the variance by default
        ica = mne.preprocessing.ICA(
            max_iter='auto', # n-1
            # use ‘extended infomax’ method for fitting the ICA, to comply with the ICLabel requirements
            method = 'infomax', 
            fit_params = dict(extended=True),
            random_state = 42,
        )

        # run ica on epochs or continuous
        if ica_input_type=="epoch":
            # get event info for segmentation
            events_from_annot, event_dict = mne.events_from_annotations(raw_filt, verbose='WARNING')
            # segmentation for ERP
            epochs = mne.Epochs(
                raw_filt,
                events = events_from_annot, event_id = event_dict,
                tmin = erp_t_start, tmax = erp_t_end,
                # apply baseline correction
                baseline = None,
                # remove epochs that meet the rejection criteria
                reject = None,
                preload = True,
            )
            # set ica_input
            ica_input = epochs
        else:
            ica_input = raw_filt
            
        #### get ica solution ####
        ica.fit(ica_input, picks = ['eeg'])
        #### ICLabel ####
        ic_labels = label_components(ica_input, ica, method="iclabel")
        
        # save ica solutions
        ica.save(output_dir + file.split('.')[0]+ '_icaSolution.fif', overwrite=True)

        # save
        with open(output_dir + file.split('.')[0]+ '_icLabels.pickle', 'wb') as f:
            pickle.dump(ic_labels, f)

        #### auto select brain AND other ####
        labels = ic_labels["labels"]
        exclude_idx = [
            idx for idx, label in enumerate(labels) if idx<ic_upto and label not in ["brain", "other"]
        ]
        
        # ica.apply() changes the Raw object in-place
        ica.apply(raw, exclude=exclude_idx)

        # record the bad ICs in bad_ICs.txt
        with open(output_dir + '/bad_ICs.txt', 'a+') as f:
            _ = f.write(file + '\t' + str(exclude_idx) + '\n')
        
        # save data after ICA
        raw.save(output_dir + file.split('.')[0]+ '_ica.fif')

Opening raw data file /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/2_bad_channel_corrected/Yas_258_corr_prep.fif...


/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2083397926.py:11: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/2_bad_channel_corrected/Yas_258_corr_prep.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(input_dir + file, preload=True)


    Range : 23135 ... 918784 =     92.540 ...  3675.136 secs
Ready.
Reading 0 ... 895649  =      0.000 ...  3582.596 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 825 samples (3.300 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Fitting ICA to data using 30 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 23 components
Computing Extended Infomax ICA
Fitting ICA took 140.7s.
Writing ICA solution to /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_258_corr_prep_icaSolution.fif...
Applying ICA to Raw instance
    Transforming to ICA space (23 components)
    Zeroing out 6 ICA components
    Projecting back using 30 PCA components


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 825 samples (3.300 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Fitting ICA to data using 30 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 27 components
Computing Extended Infomax ICA
Fitting ICA took 389.5s.
Writing ICA solution to /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_259_corr_prep_icaSolution.fif...
Applying ICA to Raw instance
    Transforming to ICA space (27 components)
    Zeroing out 4 ICA components
    Projecting back using 30 PCA components
Writing /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_259_corr_prep_ica.fif


/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2083397926.py:54: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_259_corr_prep_icaSolution.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica.save(output_dir + file.split('.')[0]+ '_icaSolution.fif', overwrite=True)
/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2083397926.py:74: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_259_corr_prep_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(output_dir + file.split('.')[0]+ '_ica.fif')


Closing /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_259_corr_prep_ica.fif
[done]


# Epochs
segmenting continuous EEG into epochs
- re-reference
- segmentation

## parameters

In [8]:
# directory
input_dir = work_dir + 'data_preprocessed/3_ica/'
output_dir = work_dir + 'data_preprocessed/4_erp_epochs/'
# create a folder if the folder doesn't exist
os.makedirs(output_dir, exist_ok=True)


# Epoch window: 
erp_t_start = -0.2; erp_t_end = 0.8 # ERP: -100 ~ 700 ms
baseline = (-0.2, 0)


# criteria to reject epoch
# reject_criteria = dict(eeg = 100e-6)       # 100 µV
# reject_criteria = dict(eeg = 150e-6)       # 150 µV
reject_criteria = dict(eeg = 200e-6)       # 200 µV

In [9]:
# initialize a list for subjects with too many bad trials
too_many_bad_trial_subjects = []

# get file names
all_input = os.listdir(input_dir)
all_output = os.listdir(output_dir)


#### re-reference, and epoch ####
for file in all_input:
    
    if file.endswith("ica.fif") and (file.split('.')[0] + '_epo.fif' not in all_output):
        
        # read in data
        raw = mne.io.read_raw_fif(input_dir + file, preload = True)
        
        # average-mastoids re-reference
        raw.set_eeg_reference(ref_channels = ['TP9', 'TP10'])
        
        #### this is for source calculation ####
        # filter the data, optional
        # raw = raw.filter(l_freq=None, h_freq=30) 

        # sphere = mne.make_sphere_model('auto', 'auto', raw.info)
        # src = mne.setup_volume_source_space(sphere=sphere, exclude=30., pos=15.)
        # forward = mne.make_forward_solution(raw.info, trans=None, src=src, bem=sphere)
        # raw = raw.set_eeg_reference('REST', forward=forward)
        ########################################

        # pick EEG channels
        # picks = mne.pick_types(raw.info, eeg = True)
        
        # get event info for segmentation
        events_from_annot, event_dict = mne.events_from_annotations(raw, verbose='WARNING')
        
        # segmentation for ERP
        epochs = mne.Epochs(
            raw,
            events = events_from_annot, event_id = event_dict,
            tmin = erp_t_start, tmax = erp_t_end,
            # apply baseline correction
            baseline = baseline,
            # remove epochs that meet the rejection criteria
            reject = reject_criteria,
            preload = True,
        )

        # remove 0 trial events, record info, and check if a subject is bad
        for k, v in event_dict.items():

            # good trial count
            trial_count = len(epochs[k])

            # remove 0 trial event
            if trial_count==0:
                del epochs.event_id[k]

            # good trial rate
            goodTrial_rate = round( trial_count/sum(events_from_annot[:,2]==v), 2 )
            
            # record epoch summary
            with open(output_dir + 'epoch_summary.txt', 'a+') as f:
                _ =f.write(file.split('_')[1] + '\t' + k + '\t' + str(trial_count) + '\t' + str(goodTrial_rate) + '\n')
                
            # mark a participant bad if any condition has fewer than 1/2 trials
            if ( goodTrial_rate < 0.5 ):
                # mark the subject file as bad
                if file.split('_')[1] not in too_many_bad_trial_subjects:
                    too_many_bad_trial_subjects.append(file.split('_')[1])

        # save single participant file
        epochs.save(output_dir + file.split('.')[0] + '_epo.fif',
                   overwrite=True)


# export the record of bad subjects for ERP
with open(output_dir + 'too_many_bad_trial_subjects.txt', 'w') as file:
    # Write each item in the list to the file
    for item in too_many_bad_trial_subjects:
        file.write(item + '\n')

Opening raw data file /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_258_corr_prep_ica.fif...


/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2338549203.py:15: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_258_corr_prep_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(input_dir + file, preload = True)


    Range : 23135 ... 918784 =     92.540 ...  3675.136 secs
Ready.
Reading 0 ... 895649  =      0.000 ...  3582.596 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
1714 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1714 events and 251 original time points ...
    Rejecting  epoch based on EEG : ['O2']
    Rejecting  epoch based on EEG : ['CP5']
    Rejecting  epoch based on EEG : ['O2']
    Rejecting  epoch based on EEG : ['FC5', 'C3', 'T7', 'CP5', 'P4', 'CP6', 'C4', 'T8', 'FT10', 'FC6', 'F4', 'F8']
    Rejecting  epoch based on EEG : ['FC5', 'CP6', 'C4', 'T8', 'FT10', 'FC6', 'F8']
    Rejecting  epoch based on EEG : ['CP6', 'C4', 'FT10', 'F4', 'F8']
    Rejecting  epoch based on EEG : ['F3', 'FC5', 'C3', 'T7', 'CP5', 'CP6', 'C4', 'T8', 'FT10', 'FC6', 'F8']
    Rejecting  epoch based on EEG : ['CP6', 'C4', 'FT10', 'FC6', 'F4', 'F8']
    Rejec

/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2338549203.py:15: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_259_corr_prep_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(input_dir + file, preload = True)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
1373 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1373 events and 251 original time points ...
    Rejecting  epoch based on EEG : ['CP6']
1 bad epochs dropped
Opening raw data file /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_263_corr_prep_ica.fif...
    Range : 509 ... 897022 =      2.036 ...  3588.088 secs
Ready.


...

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
1668 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1668 events and 251 original time points ...
    Rejecting  epoch based on EEG : ['Fz', 'FC1']
1 bad epochs dropped
Opening raw data file /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_260_corr_prep_ica.fif...
    Range : 683 ... 869409 =      2.732 ...  3477.636 secs
Ready.
Reading 0 ... 868726  =      0.000 ...  3474.904 secs...


/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2338549203.py:15: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_260_corr_prep_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(input_dir + file, preload = True)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
1657 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1657 events and 251 original time points ...
0 bad epochs dropped
Opening raw data file /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_265_corr_prep_ica.fif...
    Range : 0 ... 838511 =      0.000 ...  3354.044 secs
Ready.
Reading 0 ... 838511  =      0.000 ...  3354.044 secs...


/var/folders/j8/8nyf6njj5nzfjcv6s23y3nn40000gn/T/ipykernel_65641/2338549203.py:15: RuntimeWarning: This filename (/Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/3_ica/Yas_265_corr_prep_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(input_dir + file, preload = True)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
1657 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1657 events and 251 original time points ...
    Rejecting  epoch based on EEG : ['CP6', 'C4', 'FT10']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['P7']
    Rejecting  epoch based on EEG : ['CP6', 'C4', 'FT10']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['CP6', 'C4', 'FT10']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['O1']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['CP6', 'C4']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['FT9', 'T7']
    Rejecting  epoch based on EEG : ['C4']
    Rejecting  epoch based on EEG : ['CP6', 'C4']
    Reje

# Evokeds

## parameters

In [10]:
# directory
input_dir = work_dir + 'data_preprocessed/4_erp_epochs/'
output_dir = work_dir + 'data_preprocessed/5_averaged/'
# create a folder if the folder doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [11]:
#### get ERP ####

# get file names
all_input = os.listdir(input_dir)
all_output = os.listdir(output_dir)

# initialize a dictionary to store ERPs
all_evokeds = {}

# for each file
for file in all_input:
    
    if file.endswith("_epo.fif"):
        
        # extract subject number
        ppt = file.split('_')[1]

        # initialize dictionary for single-subject ERP
        all_evokeds[ppt] = {}
        
        # read in data
        epochs = mne.read_epochs(input_dir + file, preload = True)

        # average | get ERP for each condition
        evoked = epochs.average(by_event_type=True)

        # initialize dictionary for single-participant ERP
        all_evokeds[ppt] = {}

        # add key for each condition for analysis
        for cond in evoked:
            # append the evoked data to the dictioncary of evoked data
            all_evokeds[ppt][cond.comment] = cond

        # reduce memory usage
        del epochs


# Saving the ERP data:
with open(output_dir + '/all_evokeds.pkl', 'wb') as f:
    pickle.dump(all_evokeds, f)
del all_evokeds

Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_epochs/Yas_152_corr_prep_ica_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1675 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_epochs/Yas_200_corr_prep_ica_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1426 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_epochs/Yas_216_corr_prep_ica_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 C

# ERSP
segmenting continuous EEG into epochs for ERSP analysis
- re-reference
- segmentation

## parameters

In [135]:
# directory
input_dir = work_dir + 'data_preprocessed/3_ica/'
output_dir = work_dir + 'data_preprocessed/4_ersp/'
# create a folder if the folder doesn't exist
os.makedirs(output_dir, exist_ok=True)


# Epoch window: 
epoch_t_start = -1; epoch_t_end = 2

# baseline
epoch_baseline = None
tfr_baseline = (-0.45, -0.3)

# criteria to reject epoch
# reject_criteria = dict(eeg = 100e-6)       # 100 µV
# reject_criteria = dict(eeg = 150e-6)       # 150 µV
reject_criteria = dict(eeg = 200e-6)       # 200 µV


tfr_method = 'morlet'

# frequencies
freq_start = 3
freq_end = 30
n_freq = 28
freqs = np.linspace(start=freq_start, stop=freq_end, num=n_freq)

# cycles
cycl_start = 3
cycl_step = 0.8
n_cycl = n_freq
n_cycles = np.linspace(start=cycl_start, stop=cycl_start+cycl_step*(n_cycl-1), num=n_cycl)

condition_list = ['ChEn-stan', 'ChEn-devi', 'EnCh-stan', 'EnCh-devi']


# English participants
with open('ppts_ENG.txt', 'r') as f:
    ENG_ppts = [line.strip() for line in f]

# Chinese participants
with open('ppts_CHI.txt', 'r') as f:
    CHI_ppts = [line.strip() for line in f]
    
# South Asia participants
with open('ppts_SOME.txt', 'r') as f:
    SOME_ppts = [line.strip() for line in f]

# make a dictionary for participants
ppt_dict = {'ENG': ENG_ppts, 'CHI': CHI_ppts, 'SOME': SOME_ppts}

# subjects to exclude
exclude_ppts = [
    '140', # no data
    '197', # no data
    
    '135', # incomplete data
    '179', # incomplete data
    '182', # incomplete data

    '141', # wrong experiment was run

    '261', # no StimTrak recording

    '115', # unknown

    '187', # too many bad stims
    '200', # too many bad stims
    '259', # too many bad stims

    '268', # not included yet
    '269', # not included yet
    '270', # not included yet
]

In [126]:
# initialize a list for subjects with too many bad trials
too_many_bad_trial_participants = []

# get file names
all_input = os.listdir(input_dir)
all_output = os.listdir(output_dir)


#### re-reference, and epoch ####
for file in all_input:
    
    if file.endswith("ica.fif") and (file.split('.')[0] + '_tfr.hdf5' not in all_output):
        
        # read in data
        raw = mne.io.read_raw_fif(input_dir + file, preload = True)
        
        # average-mastoids re-reference
        raw.set_eeg_reference(ref_channels = ['TP9', 'TP10'])
        
        #### this is for source calculation ####
        # filter the data, optional
        # raw = raw.filter(l_freq=None, h_freq=30) 

        # sphere = mne.make_sphere_model('auto', 'auto', raw.info)
        # src = mne.setup_volume_source_space(sphere=sphere, exclude=30., pos=15.)
        # forward = mne.make_forward_solution(raw.info, trans=None, src=src, bem=sphere)
        # raw = raw.set_eeg_reference('REST', forward=forward)
        ########################################

        # pick EEG channels
        # picks = mne.pick_types(raw.info, eeg = True)
        
        # get event info for segmentation
        events_from_annot, event_dict = mne.events_from_annotations(raw, verbose='WARNING')
        
        # segmentation for ERP
        epochs = mne.Epochs(
            raw,
            events = events_from_annot, event_id = event_dict,
            tmin = epoch_t_start, tmax = epoch_t_end,
            # apply baseline correction
            baseline = epoch_baseline,
            # remove epochs that meet the rejection criteria
            reject = reject_criteria,
            preload = True,
        )
        


        # for each event, remove 0 trial events, record info, and check if a subject is bad
        for k, v in event_dict.items():

            # good trial count
            trial_count = len(epochs[k])

            # remove 0 trial event
            if trial_count==0:
                del epochs.event_id[k]

            # good trial rate
            goodTrial_rate = round( trial_count/sum(events_from_annot[:,2]==v), 2 )
            
            # record epoch summary
            with open(output_dir + 'epoch_summary.txt', 'a+') as f:
                _ =f.write(file.split('_')[1] + '\t' + k + '\t' + str(trial_count) + '\t' + str(goodTrial_rate) + '\n')
                
            # mark a participant bad if any condition has fewer than 1/2 trials
            if ( goodTrial_rate < 0.5 ):
                # mark the subject file as bad
                if file.split('_')[1] not in too_many_bad_trial_participants:
                    too_many_bad_trial_participants.append(file.split('_')[1])


        # ERSP

        # initialize tfrs list
        tfrs = []
        
        for condition in condition_list:
            # compute ERSP
            power = epochs[condition].compute_tfr(
                method=tfr_method,
                freqs=freqs,
                n_cycles=n_cycles,
                # return_itc=True,
                decim=3,
                # average=True,
            )
            
            # # normalization
            # # power.apply_baseline(mode="logratio", baseline=tfr_baseline)
            # power.apply_baseline(mode="ratio", baseline=tfr_baseline)
            
            # average
            power = power.average(method = 'mean', dim = 'epochs')
            
            # add comment
            power.comment = condition

            tfrs.append(power)
            
        # save single subject file
        mne.time_frequency.write_tfrs(fname=output_dir + file.split('.')[0] + '_tfr.hdf5', 
                                      tfr=tfrs)

        # release memory
        del power, tfrs


# export the record of bad subjects for ERP
with open(output_dir + 'too_many_bad_trial_participants.txt', 'w') as file:
    # Write each item in the list to the file
    for item in too_many_bad_trial_participants:
        file.write(item + '\n')

In [133]:
# grand average

# directory
input_dir = work_dir + 'data_preprocessed/4_ersp/'
output_dir = work_dir + 'data_analysis/'
# create a folder if the folder doesn't exist
# os.makedirs(output_dir, exist_ok=True)

condition_list = ['ChEn-stan', 'ChEn-devi', 'EnCh-stan', 'EnCh-devi']

# get file names
all_input = os.listdir(input_dir)


# for each group
for ppt_group in ['ENG', 'CHI', 'SOME']:

    # for each condition
    for condition in condition_list:
        
        # initialize ERSP list
        tfrs = []

        for file in all_input:
    
            if file.endswith("tfr.hdf5") and (file.split('_')[1] in ppt_dict[ppt_group]):

                # exclude bad data
                if file.split('_')[1] in exclude_ppts:
                    continue
                
                # read in data
                power = mne.time_frequency.read_tfrs(fname=input_dir + file, condition=condition)

                # append
                tfrs.append(power)

        # create a list of grand average differences
        tfrs = mne.grand_average(tfrs)

        # save single subject file
        mne.time_frequency.write_tfrs(fname=output_dir + 'ersp_gav_' + ppt_group + '_' + condition + '.hdf5', 
                                      tfr=tfrs, overwrite=True)
        

Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_ersp/Yas_156_corr_prep_ica_tfr.hdf5 ...
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_ersp/Yas_123_corr_prep_ica_tfr.hdf5 ...
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_ersp/Yas_167_corr_prep_ica_tfr.hdf5 ...
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_ersp/Yas_124_corr_prep_ica_tfr.hdf5 ...
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_ersp/Yas_152_corr_prep_ica_tfr.hdf5 ...
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/Yas accent/data_preprocessed/4_ersp/Yas_163_corr_prep_ica_tfr.hdf5 ...
Reading /Users/chaohan/Library/CloudStorage/OneDrive-UniversityofToronto/Projects/